In [2]:
import openpyxl
import re

In [3]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

In [6]:
workbook = openpyxl.load_workbook('tesla_2023_all_table.xlsx')

# Get the sheet names
sheet_names = workbook.sheetnames

In [8]:
len(sheet_names)

58

In [9]:
df = pd.read_excel('tesla_2023_all_table.xlsx', sheet_name = sheet_names[0])

In [10]:
df.head(9)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22
0,NaN,NaN,country,original_file_name,dtb_ready_name,company_id,redundant_text,table_id,page_header_text,toc_text,...,group_id,unit,source_unit,base_factor,date,latest_year,Empty table,Qualitative table,file_page_number,note_number
1,NaN,NaN,us,NaN,tesla,tesla,NaN,revenues,NaN,NaN,...,NaN,usd,ccy,1000000,2023_01_01,T,False,False,39,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,0,1,2,3,4,5,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,0,(Dollars in millions),Year Ended December 31,Year Ended December 31,Year Ended December 31,2023 vs. 2022 Change,2023 vs. 2022 Change,2022 vs. 2021 Change,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,1,(Dollars in millions),2023,2022,2021,$,%,$,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,2,Automotive sales,$ 78509 $,67210,$ 44125 $,11299,17 %,$ 23085,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,3,Automotive regulatory credits,1790,1776,1465,14,1 %,311,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,4,Automotive leasing,2120,2476,1642,-356,-14%,834,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
df.iloc[1,7]

'revenues'

In [15]:
quant = pd.read_csv('tesla_quantitative.csv')
quant.head()

,primary_key,date_last_updated,published_date,reported_date,doc_page_num,file_page_num,table_id,country,geo_1_id,geo_1_name,...,2015_check,2016_check,2017_check,2018_check,2019_check,2020_check,2021_check,2022_check,2023_check,Remarks
0,1,2024-02-05T14:41:33+00:00,2023-12-31T00:00:00+00:00,2023-12-31T00:00:00+00:00,57.0,57.0,revenue_by_source,us,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,2024-02-05T14:41:33+00:00,2023-12-31T00:00:00+00:00,2023-12-31T00:00:00+00:00,57.0,57.0,revenue_by_source,us,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,2024-02-05T14:41:33+00:00,2023-12-31T00:00:00+00:00,2023-12-31T00:00:00+00:00,57.0,57.0,revenue_by_source,us,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,2024-02-05T14:41:33+00:00,2023-12-31T00:00:00+00:00,2023-12-31T00:00:00+00:00,57.0,57.0,revenue_by_source,us,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,2024-02-05T14:41:33+00:00,2023-12-31T00:00:00+00:00,2023-12-31T00:00:00+00:00,57.0,57.0,revenue_by_source,us,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
quant = pd.read_csv('tesla_quantitative.csv')
table_id = df.iloc[1,7]
temp_quant = quant[quant['table_id'] == table_id]
# temp_quant.head()
metric_names = temp_quant['metric_name']

In [62]:
# Cleans and fetches the TS removing the change columns
years = [2020, 2021, 2022]

def filter_strings(strings, years):
    filtered_strings = []

    # Create a regex pattern using the years list
    pattern = r"(?<!\d)(?:{})(?!\d)".format("|".join(str(year) for year in years))

    for string in strings:
        # Check if the string matches the pattern
        match = re.findall(pattern, string)
        if match and len(set(match)) == 1 and "vs" not in string.lower() and "change" not in string.lower():
            filtered_strings.append(string)

    return filtered_strings

In [63]:
# removes special characters from values under the TS
def remove_special_characters(strings):
    cleaned_strings = []

    for string in strings:
        # Use regex to remove non-digit and non-negative sign characters
        cleaned_string = re.sub(r'[^-.\d]', '', string)
        cleaned_strings.append(cleaned_string)

    return cleaned_strings

In [64]:
# clean the years to match with quant file
def clean_years(unclean_years):
    clean_years = []
    #Create a regex pattern using uncleaned years
    pattern=r'\b\d{4}\b'
    for value in unclean_years:
        matches = re.findall(pattern, value)
        clean_years.append(matches[0])
    return clean_years

In [65]:
def cos_sim(X, Y, similarity):
    '''
    This function calculates the cosine similarity between two strings.
    
    Arg: X_list (type - list [Tokenized string])
         Y (type - string [Raw string from filtered dtb/individual qual])
         j (type - int [index of filtered dtb/individual qual]) and
         similarity (type - dictinary [Dictionary with key as index of source and value as similarity value])
    
    Returns: None 
    '''
    # Tokenize X_list if it's not already tokenized
    #if isinstance(X_list[0], str):
    #    X_list = [' '.join(X_list)]

    # Create CountVectorizer object and fit to the combined texts
    #vectorizer = CountVectorizer().fit_transform([X] + [Y])
    
    # Create TfidfVectorizer object and fit to the combined texts
    vectorizer = TfidfVectorizer().fit_transform([X] + [Y])
    
    # Calculate cosine similarity between X and Y
    cosine = vectorizer[:-1].dot(vectorizer[-1].T).toarray()[0][0]

    # Add similarity score to dictionary
    key = Y
    value = cosine
    dict_to_append = {key: value}
    similarity.update(dict_to_append)

In [66]:
# case for taking average of values

for i in range(4,7): #gives unclean TS years
    row_val = list(df.iloc[i])
    row_val_cl = list(pd.Series(row_val).dropna())
    row_val_cl = [str(value) for value in row_val_cl]
    filtered = filter_strings(row_val_cl, years)
    #print(filtered)
    if len(filtered)>0:
        break
    
years = filtered
years_cleaned = clean_years(years)
year_dict =  {}
for y, yc in zip(years, years_cleaned): #to get a list of values under each TS
    year_ind = list(df.iloc[i]).index(y)
    year_val = list(df.iloc[i+1:, year_ind])
    year_val = [str(value) for value in year_val]
    year_val_cleaned = remove_special_characters(year_val)
    year_val_cleaned = [item for item in year_val_cleaned if item != '' and item != '0.0'] #removes blanks
    key = yc
    value = year_val_cleaned
    dict_to_append = {key: value}
    year_dict.update(dict_to_append)
        
if len(np.unique(np.array(metric_names))) == len(metric_names):
    for key in year_dict.keys():
        quant_val = pd.Series(list(temp_quant[key])).astype(float).mean()
        print("Quant file average: ",quant_val)
        all_tables_val = pd.Series(year_dict[key]).astype(float).mean()
        print("All tables average: ",all_tables_val)
        if quant_val == all_tables_val:
            print(key, ": Pass")
        else:
            print(key, ": Fail")
else:
    print('step 2')

step 2


In [67]:
# No dims | only year present
metric_ids_alltable = df.iloc[i+1:,3]
for X in metric_ids_alltable:
    similarity = {}
    print(X)
    for Y in metric_names:
        cos_sim(X,Y,similarity)
    max_sim = max(similarity.values())
    if str(int(temp_quant['2022'].iloc[(list(similarity.values()).index(max_sim))])) == list(year_dict['2022'])[list(metric_ids_alltable).index(X)]:
        print('yes') #problem arises when both are in diff format
    else:
        print('no')

nan


ValueError: np.nan is an invalid document, expected byte or unicode string.

In [68]:
# tables with: dims on top | year on top

for j in range(4,7):
    for dim in list(df.iloc[j].str.title()):
        metric_ids_alltable = df.iloc[j+1:,3]
        if dim in list(np.unique(temp_quant['dim_1_name'])):
            dim_val = list(df.iloc[j+1:, list(df.iloc[j].str.title()).index(dim)])
            dim_val = [str(value) for value in dim_val]
            dim_val_cleaned = remove_special_characters(dim_val)
            dim_val_cleaned = pd.Series(dim_val_cleaned)
            dim_val_cleaned = dim_val_cleaned.replace(to_replace = '', value = 0)
            temp_quant_new = temp_quant[temp_quant['dim_1_name'] == dim]
            temp_quant_new_col = temp_quant_new[years_cleaned[list(df.iloc[j].str.title()).index(dim)-np.array(row_val.index(years[0]))]].fillna(0)
            # above subtracted first year index from dim index to find the index of year in years_cleaned list
            for X in metric_ids_alltable:
                similarity = {}
                for Y in temp_quant_new['metric_name']:
                    cos_sim(X,Y,similarity)
                max_sim = max(similarity.values())
                print(dim)
                #print(similarity)
                if str(int(temp_quant_new_col.iloc[(list(similarity.values()).index(max_sim))])) == str(list(dim_val_cleaned)[list(metric_ids_alltable).index(X)]):
                    print('yes',list(dim_val_cleaned)[list(metric_ids_alltable).index(X)]) #problem arises when both are in diff format
                else:
                    print('no',list(dim_val_cleaned)[list(metric_ids_alltable).index(X)],X)
        else:
            continue

Adjusted Cost
yes 13965
Adjusted Cost
yes 2188
Adjusted Cost
yes 897
Adjusted Cost
yes 907
Adjusted Cost
yes 4252
Adjusted Cost
no 0 Total cash, cash equivalents and
Adjusted Cost
yes 22209
Gross Unrealized Gains
yes 0
Gross Unrealized Gains
yes 0
Gross Unrealized Gains
yes 0
Gross Unrealized Gains
yes 0
Gross Unrealized Gains
yes 1
Gross Unrealized Gains
no 0 Total cash, cash equivalents and
Gross Unrealized Gains
yes 1
Gross Unrealized Losses
yes 0
Gross Unrealized Losses
yes 0
Gross Unrealized Losses
yes -3
Gross Unrealized Losses
yes -22
Gross Unrealized Losses
yes 0
Gross Unrealized Losses
no 0 Total cash, cash equivalents and
Gross Unrealized Losses
yes -25
Fair Value
yes 13965
Fair Value
yes 2188
Fair Value
yes 894
Fair Value
yes 885
Fair Value
yes 4253
Fair Value
no 0 Total cash, cash equivalents and
Fair Value
yes 22185
Cash And Cash Equivalents
yes 13965
Cash And Cash Equivalents
yes 2188
Cash And Cash Equivalents
yes 0
Cash And Cash Equivalents
yes 0
Cash And Cash Equivalent

In [52]:
#case: dims present on top | dont repeat | year not present hence all data in latest year
for j in range(4,7):
    for dim in list(np.unique(temp_quant['dim_1_name'].dropna())):
        metric_ids_alltable = df.iloc[j+1:,3]
        if dim in list(df.iloc[j].str.title()):
            dim_val = list(df.iloc[j+1:, list(df.iloc[j].str.title()).index(dim)])
            dim_val = [str(value) for value in dim_val]
            dim_val_cleaned = remove_special_characters(dim_val)
            dim_val_cleaned = pd.Series(dim_val_cleaned)
            dim_val_cleaned = dim_val_cleaned.replace(to_replace = '', value = 0)
            temp_quant_new = temp_quant[temp_quant['dim_1_name'] == dim]
            temp_quant_new_col = temp_quant_new['2022'].fillna(0)
            for X in metric_ids_alltable:
                similarity = {}
                for Y in temp_quant_new['metric_name']:
                    cos_sim(X,Y,similarity)
                max_sim = max(similarity.values())
                print(dim)
                #print(similarity)
                if str(int(temp_quant_new_col.iloc[(list(similarity.values()).index(max_sim))])) == str(list(dim_val_cleaned)[list(metric_ids_alltable).index(X)]):
                    print('yes',list(dim_val_cleaned)[list(metric_ids_alltable).index(X)]) #problem arises when both are in diff format
                else:
                    print('no',list(dim_val_cleaned)[list(metric_ids_alltable).index(X)],X)
    if dim in list(df.iloc[j]):
        break

In [53]:
# case when dim present in metrics | only latest year
dim2_list = []
metric_ids_alltable = df.iloc[i+1:,3]
for m in range(1,len(metric_ids_alltable)+1):
    if df.iloc[i+m,3] == df.iloc[i+m,4]:
        dim2 = df.iloc[i+m,3].title()
        dim2_list.append(dim2)
    else:
        continue

In [54]:
list(df.iloc[list(df.iloc[i+1:,3].str.title()).index(dim)+i+2:,3].str.title()).index(x)

NameError: name 'x' is not defined

In [55]:
float(col_val_cleaned[list(df.iloc[i+1:,3].str.title()).index(dim)+list(df.iloc[list(df.iloc[i+1:,3].str.title()).index(dim)+i+2:,3].str.title()).index(x)+i+2])

NameError: name 'col_val_cleaned' is not defined

In [56]:
col_val_cleaned

NameError: name 'col_val_cleaned' is not defined

In [57]:
# Case when dim present in metric | year present on top
for dim in list(np.unique(temp_quant['dim_1_name'].dropna())):
    if dim in dim2_list:
        temp_quant_new = temp_quant[temp_quant['dim_1_name'] == dim]
        metric_id_new = df.iloc[list(df.iloc[:,3].str.title()).index(dim):,3].str.title()
        for x in temp_quant_new['metric_name']:
            if x in list(df.iloc[i+1:,3].str.title()):
                for y in years_cleaned:
                    temp_quant_new_metric = list(temp_quant_new[y])
                    col_val = df.iloc[:,list(df.iloc[i,:]).index(years[0])+years_cleaned.index(y)]
                    col_val = [str(value) for value in col_val]
                    col_val_cleaned = remove_special_characters(col_val)
                    col_val_cleaned = pd.Series(col_val_cleaned)
                    col_val_cleaned = col_val_cleaned.replace(to_replace = '', value = 0)
                    #taken the location from dim to end and from that list taken the location of metric to be found and matched the corresponding value
                    if float(col_val_cleaned[list(df.iloc[i+1:,3].str.title()).index(dim)+list(df.iloc[list(df.iloc[i+1:,3].str.title()).index(dim)+i+2:,3].str.title()).index(x)+i+2]) == temp_quant_new_metric[list(temp_quant_new['metric_name']).index(x)]:
                        print('yay', x, y,float(col_val_cleaned[list(df.iloc[i+1:,3].str.title()).index(dim)+list(df.iloc[list(df.iloc[i+1:,3].str.title()).index(dim)+i+2:,3].str.title()).index(x)+i+2]))
                    else:
                        print('noo', x, y)
            else:
                print('no',x)
    else:
        print('no', dim)